# This code book is modified and shortened from the original version which you can find here:
https://cloud.google.com/ai-platform/docs/getting-started-keras

* This will show you how to use the AI Platform in GCP to train, store, and serve predictions from a Keras model.

* This is best run in Colab, although you may also be able to use notebooks provided on the AI platform. In this live assignment please just run it in Colab.

* In order for this to work, the gmail that you use to run the Colab should have been added to the Fourthbrain GCP project (This has already been done for students in this cohort).

In [ ]:
# Copyright 2019 Google LLC
# Redo for region= US-east1
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting started: Training and prediction with Keras in AI Platform

## Overview

This tutorial shows how to train a neural network on the GCP AI Platform
using the Keras sequential API and how to serve predictions from that
model.

Keras is a high-level API for building and training deep learning models.
[tf.keras](https://www.tensorflow.org/guide/keras) is TensorFlow’s
implementation of this API.

The first two parts of the tutorial walk through training a model on Cloud
AI Platform using prewritten Keras code, deploying the trained model to
AI Platform, and serving online predictions from the deployed model.

The last part of the tutorial digs into the training code used for this model and ensuring it's compatible with AI Platform. To learn more about building
machine learning models in Keras more generally, read [TensorFlow's Keras
tutorials](https://www.tensorflow.org/tutorials/keras).

### Dataset

This tutorial uses the [United States Census Income
Dataset](https://archive.ics.uci.edu/ml/datasets/census+income) provided by the
[UC Irvine Machine Learning
Repository](https://archive.ics.uci.edu/ml/index.php). This dataset contains
information about people from a 1994 Census database, including age, education,
marital status, occupation, and whether they make more than $50,000 a year.

### Objective

The goal is to train a deep neural network (DNN) using Keras that predicts
whether a person makes more than $50,000 a year (target label) based on other
Census information about the person (features).

This tutorial focuses more on using this model with AI Platform than on
the design of the model itself. However, it's always important to think about
potential problems and unintended consequences when building machine learning
systems. See the [Machine Learning Crash Course exercise about
fairness](https://developers.google.com/machine-learning/crash-course/fairness/programming-exercise)
to learn about sources of bias in the Census dataset, as well as machine
learning fairness more generally.

### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
# The path below should point to the directory containing this notebook in Google Drive. We'll also clone the cloudml-samples GitHub repository at this location
# Change it if necessary
os.chdir('/content/drive/MyDrive/FourthBrain/Week 13')
!ls

cloudml-samples  week13_V1.ipynb


### Get some code and dependencies

First, download the training code and change the notebook's working directory. The code and data get downloaded to your GDrive:

In [3]:
# Clone the repository of AI Platform samples
! git clone --depth 1 https://github.com/GoogleCloudPlatform/cloudml-samples

fatal: destination path 'cloudml-samples' already exists and is not an empty directory.


In [4]:
# Set the working directory to the sample code directory
%cd cloudml-samples/census/tf-keras

/content/drive/MyDrive/FourthBrain/Week 13/cloudml-samples/census/tf-keras


Notice that the training code is structured as a Python package in the
`trainer/` subdirectory:

In [5]:
# `ls` shows the working directory's contents. The `p` flag adds trailing 
# slashes to subdirectory names. The `R` flag lists subdirectories recursively.
! ls -pR

.:
hptuning_config.yaml  README.md  requirements.txt  scripts/  trainer/

./scripts:
train-local.sh

./trainer:
__init__.py  model.py  task.py	util.py


Run the following cell to install Python dependencies needed to train the model locally. When you run the training job in AI Platform,
dependencies are preinstalled based on the [runtime
version](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list)
you choose.

In [6]:
! pip install -r requirements.txt

     |████████████████████████████████| 110.5 MB 1.1 kB/s 
     |████████████████████████████████| 20.1 MB 84.7 MB/s 
     |████████████████████████████████| 3.8 MB 38.4 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 503 kB 60.4 MB/s 
     |████████████████████████████████| 2.9 MB 47.1 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=8b1e547927669b4ee1b1cc600ab92b45d25f02ec3bace8776ec7fbcdc736cd8e
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: ten

^ If this tells you that you have to restart the runtime for these installs to take effect, then do this now. You'll have to re-run the cells that change the working directory

## Setting up your GCP project

**Steps 1 and 2 below are required only if you're setting up a NEW "project" on the Google Cloud Platform. You don't have to do these because you've already been added to a Fourthbrain GCP project (it's called 'FB-MLE-Jul-21')**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager)

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project). 

**👇 For the Live assignment start from here.**

3. [Enable the AI Platform ("Cloud Machine Learning Engine") and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. Enter the project ID in the cell below. It should already be populated for you as "fb-mle-jul-21", which is the project we're using for the July 2021 MLE cohort. Run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook. 

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [6]:
PROJECT_ID = "fb-mle-jul-21" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**
https://cloud.google.com/storage/docs/creating-buckets#storage-create-bucket-console
When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. AI Platform runs
the code from this package. In this tutorial, AI Platform also saves the
trained model that results from your job in the same bucket. You can then
create an AI Platform model version based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. Pick something that doesn't conflict with other students' buckets. [Here is a guide on naming storage buckets](https://cloud.google.com/storage/docs/naming-buckets#:~:text=Bucket%20names%20reside%20in%20a,responds%20with%20an%20error%20message.). 

In [8]:
BUCKET_NAME = "week13-ml-bucket" #@param {type:"string"}
REGION = "us-east1" #@param {type:"string"}

# Click the link below to create your storage bucket. Give it the same name you chose in the cell above:
https://cloud.google.com/storage/docs/creating-buckets


In [11]:
#This will check connection and list the connected data bucket. Ensure Bitbucket created before this.
from google.colab import auth
auth.authenticate_user()
project_id = PROJECT_ID
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://almicias-first-bucket/
gs://artifacts.fb-mle-jul-21.appspot.com/
gs://fg_liveassignment_week13/
gs://gs-colab-week-13-live-assignment/
gs://jt-4bai-gcp-bucket/
gs://rabiaa_bucket/
gs://shizhao-lu-week-13-live-assignment/
gs://sk-adi-unique-bucket/
gs://sk-my-unique-bucket-flor/
gs://sk-my-unique-minhahwang-test/
gs://sk-my-yassine-unique-bucket-name/
gs://week13-ml-bucket/


## Part 1. Quickstart for training in AI Platform

This section of the tutorial walks you through submitting a training job to Cloud
AI Platform. This job runs sample code that uses Keras to train a deep neural
network on the United States Census data. It outputs the trained model as a
[TensorFlow SavedModel
directory](https://www.tensorflow.org/guide/saved_model#save_and_restore_models)
in your Cloud Storage bucket.

### Train your model locally

Before training on AI Platform, train the job locally to verify the file
structure and packaging is correct.

For a complex or resource-intensive job, you
may want to train locally on a small sample of your dataset to verify your code.
Then you can run the job on AI Platform to train on the whole dataset.

This sample runs a relatively quick job on a small dataset, so the local
training and the AI Platform job run the same code on the same data.

Run the following cell to train a model locally:

In [12]:
# Explicitly tell `gcloud ai-platform local train` to use Python 3 
! gcloud config set ml_engine/local_python $(which python3)

# This is similar to `python -m trainer.task --job-dir local-training-output`
# but it better replicates the AI Platform environment, especially for
# distributed training (not applicable here).
! gcloud ai-platform local train \
  --package-path trainer \
  --module-name trainer.task \
  --job-dir local-training-output

Updated property [ml_engine/local_python].
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2021-11-06 17:05:57.030905: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-06 17:05:57.035576: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-11-06 17:05:57.035912: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55c160b37640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-06 17:05:57.035954: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Train on 254 steps, validate on 1 steps

Epoch 00001:

This should have saved the trained model in a directory called `local-training-output` (which is mounted to your Google Drive):

In [13]:
!ls local-training-output/

keras_export  keras_tensorboard


### Train your model using AI Platform

Next, compare this to submitting a training job to AI Platform. This runs the training module
in the cloud and exports the trained model to Cloud Storage.

First, give your training job a name and choose a directory within your Cloud
Storage bucket for saving intermediate and output files:

In [16]:
####ENTER HERE#############################################
JOB_NAME = 'js_week13_job'
JOB_DIR = 'gs://' + BUCKET_NAME + '/keras-job-dir'
print(JOB_DIR)

gs://week13-ml-bucket/keras-job-dir


Run the following command to package the `trainer/` directory, upload it to the
specified `--job-dir`, and instruct AI Platform to run the
`trainer.task` module from that package.

The `--stream-logs` flag lets you view training logs in the cell below. You can
also see logs and other job details in the GCP Console.

Don't do this now, but you can optionally perform hyperparameter tuning by using the included
`hptuning_config.yaml` configuration file. This file tells AI Platform to tune the batch size and learning rate for training over multiple trials to maximize accuracy.

In this example, the training code uses a [TensorBoard
callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard),
which [creates TensorFlow `Summary`
`Event`s](https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter#add_summary)
during training. AI Platform uses these events to track the metric you want to
optimize. Learn more about [hyperparameter tuning in
AI Platform Training](https://cloud.google.com/ml-engine/docs/tensorflow/hyperparameter-tuning-overview).

## While you wait for this step to complete, you can check on GCP console that your job is running.

After you run the cell below, you can check on the progress of your job: https://console.cloud.google.com/ai-platform/jobs

In [17]:
! gcloud ai-platform jobs submit training $JOB_NAME \
  --package-path trainer/ \
  --module-name trainer.task \
  --region $REGION \
  --python-version 3.7 \
  --runtime-version 1.15 \
  --job-dir $JOB_DIR \
  --stream-logs

Job [js_week13_job] submitted successfully.
INFO	2021-11-06 17:07:41 +0000	service		Validating job requirements...
INFO	2021-11-06 17:07:41 +0000	service		Job creation request has been successfully validated.
INFO	2021-11-06 17:07:41 +0000	service		Waiting for job to be provisioned.
INFO	2021-11-06 17:07:41 +0000	service		Job js_week13_job is queued.
INFO	2021-11-06 17:07:43 +0000	service		Waiting for training program to start.
NOTICE	2021-11-06 17:08:16 +0000	master-replica-0.gcsfuse		Opening GCS connection...
NOTICE	2021-11-06 17:08:17 +0000	master-replica-0.gcsfuse		Mounting file system "gcsfuse"...
NOTICE	2021-11-06 17:08:17 +0000	master-replica-0.gcsfuse		File system has been successfully mounted.
INFO	2021-11-06 17:08:19 +0000	master-replica-0		Running task with arguments: --cluster={"master": ["127.0.0.1:2222"]} --task={"type": "master", "index": 0} --job={  "package_uris": ["gs://week13-ml-bucket/keras-job-dir/packages/74598a46e41649bccb4bed39e93f15207f7ac5aed06e18fe9106b211d0f

# Part 2 (Cloud Deployment). Quickstart for online predictions in AI Platform

This section shows how to use AI Platform and your trained model from Part 1
to predict a person's income bracket from other Census information about them.

### Create model and version resources in AI Platform

To serve online predictions using the model you trained and exported in Part 1,
create a *model* resource in AI Platform and a *version* resource
within it. The version resource is what actually uses your trained model to
serve predictions. This structure lets you adjust and retrain your model many times and
organize all the versions together in AI Platform. Learn more about [models
and
versions](https://cloud.google.com/ml-engine/docs/tensorflow/projects-models-versions-jobs).

First, name and create the model resource:

In [18]:
############ENTER HERE#########################
MODEL_NAME = "shim_automl_demo" #<Lastname_automl_demo>

! gcloud ai-platform models create $MODEL_NAME \
  --region $REGION

Using endpoint [https://us-east1-ml.googleapis.com/]
Created ai platform model [projects/fb-mle-jul-21/models/shim_automl_demo].


 # You can verify your job completion at the AI Training Dashboard:
 https://console.cloud.google.com/ai-platform/models/

 # Look under jobs and models on the left panel
```

Execute the following command to identify your SavedModel directory and use it to create a model version resource:

In [19]:
#ENTER HERE#####################
MODEL_VERSION = "v1"

# Get a list of directories in the `keras_export` parent directory
KERAS_EXPORT_DIRS = ! gsutil ls $JOB_DIR/keras_export/
print(KERAS_EXPORT_DIRS)

['gs://week13-ml-bucket/keras-job-dir/keras_export/', 'gs://week13-ml-bucket/keras-job-dir/keras_export/saved_model.pb', 'gs://week13-ml-bucket/keras-job-dir/keras_export/assets/', 'gs://week13-ml-bucket/keras-job-dir/keras_export/variables/']


In [20]:
# Point to the folder ../keras_export. Your index number may be different that the one here based on how many times you ahve run.
idx=0
SAVED_MODEL_PATH = KERAS_EXPORT_DIRS[idx]
print(SAVED_MODEL_PATH)

gs://week13-ml-bucket/keras-job-dir/keras_export/


In [21]:
print(MODEL_NAME)

shim_automl_demo


In [22]:
# Create model version based on that SavedModel directory
! gcloud ai-platform versions create $MODEL_VERSION \
  --model $MODEL_NAME \
  --region $REGION \
  --runtime-version 1.15 \
  --python-version 3.7 \
  --framework tensorflow \
  --origin $SAVED_MODEL_PATH

Using endpoint [https://us-east1-ml.googleapis.com/]


# Now you are ready to submit a training job to Google AI platform

# Task 2: Data Preparation for Modeling

To receive valid and useful predictions, you must preprocess input for prediction in the same way that training data was preprocessed. In a production
system, you may want to create a preprocessing pipeline that can be used identically at training time and prediction time.

For this exercise, use the training package's data-loading code to select a random sample from the evaluation data. This data is in the form that was used to evaluate accuracy after each epoch of training, so it can be used to send test predictions without further preprocessing:

In [23]:
#First, lets review the data and features that are used for classifictaion
from trainer import util

_, _, eval_x, eval_y = util.load_data()

prediction_input = eval_x.sample(20)
prediction_targets = eval_y[prediction_input.index]

prediction_input

,age,workclass,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country
7670,-0.192807,3,-0.419265,0,6,4,4,-0.144792,-0.217132,-0.679647,38
15132,-1.359762,1,-0.419265,4,2,1,4,-0.144792,-0.217132,-2.455067,38
16183,0.463605,3,-0.419265,2,0,3,4,-0.144792,-0.217132,-2.051562,38
7093,-1.359762,3,-0.419265,4,6,3,2,-0.144792,-0.217132,-0.034039,38
873,0.828278,-1,-0.419265,2,-1,0,2,-0.144792,-0.217132,0.369465,13
10762,0.682409,3,-0.030304,4,9,4,4,-0.144792,-0.217132,0.369465,38
4696,0.974147,1,-0.419265,2,7,5,1,-0.144792,-0.217132,-0.034039,-1
14240,-0.922154,3,-0.030304,4,2,3,4,-0.144792,-0.217132,0.772970,38
11856,-1.359762,3,-0.030304,4,11,3,4,-0.144792,-0.217132,-1.648058,38
11001,-1.213893,3,-0.419265,4,6,3,4,-0.144792,-0.217132,-0.034039,38


Notice that categorical fields, like `occupation`,  have already been converted to integers (with the same mapping that was used for training). Numerical fields, like `age`, have been scaled to a
[z-score](https://developers.google.com/machine-learning/crash-course/representation/cleaning-data). Some fields have been dropped from the original
data. Compare the prediction input with the raw data for the same examples:

In [24]:
#Now lets review the raw data with all features
import pandas as pd

_, eval_file_path = util.download(util.DATA_DIR)
raw_eval_data = pd.read_csv(eval_file_path,
                            names=util._CSV_COLUMNS,
                            na_values='?')

raw_eval_data.iloc[prediction_input.index]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
7670,36,Private,218188,HS-grad,9,Divorced,Machine-op-inspct,Unmarried,White,Female,0,0,32,United-States,<=50K
15132,20,Local-gov,38455,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,10,United-States,<=50K
16183,45,Private,159080,HS-grad,9,Married-civ-spouse,Adm-clerical,Own-child,White,Female,0,0,15,United-States,<=50K
7093,20,Private,273147,HS-grad,9,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
873,50,NaN,137632,HS-grad,9,Married-civ-spouse,NaN,Husband,Black,Male,0,0,45,Haiti,<=50K
10762,48,Private,59380,Some-college,10,Never-married,Prof-specialty,Unmarried,White,Female,0,0,45,United-States,<=50K
4696,52,Local-gov,181578,HS-grad,9,Married-civ-spouse,Other-service,Wife,Asian-Pac-Islander,Female,0,0,40,NaN,>50K
14240,26,Private,301298,Some-college,10,Never-married,Craft-repair,Own-child,White,Male,0,0,50,United-States,<=50K
11856,20,Private,421474,Some-college,10,Never-married,Sales,Own-child,White,Male,0,0,20,United-States,<=50K
11001,22,Private,95552,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,40,United-States,<=50K


Export the prediction input (filtered features only) to a newline-delimited JSON file:

In [25]:
import json

with open('prediction_input.json', 'w') as json_file:
  for row in prediction_input.values.tolist():
    json.dump(row, json_file)
    json_file.write('\n')

! cat prediction_input.json

[-0.19280736025465722, 3.0, -0.4192650914017433, 0.0, 6.0, 4.0, 4.0, -0.14479173735784842, -0.21713186390175285, -0.6796466218034705, 38.0]
[-1.3597621713288508, 1.0, -0.4192650914017433, 4.0, 2.0, 1.0, 4.0, -0.14479173735784842, -0.21713186390175285, -2.4550669297737535, 38.0]
[0.4636047209745767, 3.0, -0.4192650914017433, 2.0, 0.0, 3.0, 4.0, -0.14479173735784842, -0.21713186390175285, -2.0515623143259623, 38.0]
[-1.3597621713288508, 3.0, -0.4192650914017433, 4.0, 6.0, 3.0, 2.0, -0.14479173735784842, -0.21713186390175285, -0.03403923708700391, 38.0]
[0.8282780994352622, -1.0, -0.4192650914017433, 2.0, -1.0, 0.0, 2.0, -0.14479173735784842, -0.21713186390175285, 0.3694653783607877, 13.0]
[0.6824087480509881, 3.0, -0.030303770229214273, 4.0, 9.0, 4.0, 4.0, -0.14479173735784842, -0.21713186390175285, 0.3694653783607877, 38.0]
[0.9741474508195365, 1.0, -0.4192650914017433, 2.0, 7.0, 5.0, 1.0, -0.14479173735784842, -0.21713186390175285, -0.03403923708700391, -1.0]
[-0.9221541171760282, 3.0,

The `gcloud` command-line tool accepts newline-delimited JSON for online
prediction, and this particular Keras model expects a flat list of
numbers for each input example.

AI Platform requires a different format when you make online prediction requests to the REST API without using the `gcloud` tool. The way you structure
your model may also change how you must format data for prediction. Learn more
about [formatting data for online
prediction](https://cloud.google.com/ml-engine/docs/tensorflow/prediction-overview#prediction_input_data).

### Submit the online prediction request

Use `gcloud` to submit your online prediction request.

In [26]:
! gcloud ai-platform predict \
  --model $MODEL_NAME \
  --region $REGION \
  --version $MODEL_VERSION \
  --json-instances prediction_input.json

Using endpoint [https://us-east1-ml.googleapis.com/]
[[0.0182674825], [0.000166326761], [0.269278109], [0.00110688806], [0.133995712], [0.0523967743], [0.399259269], [0.0113329887], [0.00048032403], [0.00223433971], [0.000152796507], [0.000534623861], [0.956248283], [0.590556681], [0.779541731], [0.464449912], [0.313977391], [0.174936682], [0.0248344466], [0.0836561844]]


Since the model's last layer uses a [sigmoid function](https://developers.google.com/machine-learning/glossary/#sigmoid_function) for its activation, outputs between 0 and 0.5 represent negative predictions ("<=50K") and outputs between 0.5 and 1 represent positive ones (">50K").

# ...and that's about it for this tutorial! We saw how to:
1. Use the Google Cloud AI Platform to **train a model locally** (using `gcloud ai-platform local train`)
2. **Train the model as a AI Platform "job"** (using `gcloud ai-platform jobs submit training`)
3. Create a "model" asset that can then be queried with any properly-formatted input data (using `gcloud ai-platform predict`)

Yay!

## Finally, Cleaning up

You can clean up individual resources by running the following
commands:

In [27]:
# Delete model version resource
! gcloud ai-platform versions delete $MODEL_VERSION --quiet --model $MODEL_NAME 

# Delete model resource
! gcloud ai-platform models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $JOB_DIR

# If the training job is still running, cancel it
! gcloud ai-platform jobs cancel $JOB_NAME --quiet --verbosity critical

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.delete) NOT_FOUND: Field: name Error: The model resource: (695813132143, shim_automl_demo) was not found. Please create the model resource first by using 'gcloud ai-platform models create shim_automl_demo'.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: "The model resource: (695813132143, shim_automl_demo) was not found.\
      \ Please create the model resource first by using 'gcloud ai-platform models\
      \ create shim_automl_demo'."
    field: name
Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.delete) NOT_FOUND: Field: name Error: The model resource: (695813132143, shim_automl_demo) was not found. Please create the model resource first by using 'gcloud ai-platform models create shim_automl_demo'.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: "The model resource: 

In [28]:
!gsutil rm -r gs://$BUCKET_NAME

Removing gs://week13-ml-bucket/...


## What's next?

* View the [complete training
code](https://github.com/GoogleCloudPlatform/cloudml-samples/tree/master/census/tf-keras) used in this guide, which structures the code to accept custom
hyperparameters as command-line flags.
* Read about [packaging
code](https://cloud.google.com/ml-engine/docs/tensorflow/packaging-trainer) for an AI Platform training job.
* Read about [deploying a
model](https://cloud.google.com/ml-engine/docs/tensorflow/deploying-models) to serve predictions.